앞 절에서는 각 층의 활성화 값 분포를 관찰해보면, 가중치의 초깃값ㅇ르 적절히 설정하면 각 층의 활서화 값 분포가 적당히 퍼지면서 학습이 원활하게 수항됨을 배웠다. 그렇다면 각 층이 활성화를 적당히 퍼뜨리도록 "강제"해보면 어떻까? 실은 "__배치 정규화(Batch Normalization)__"가 그런 아이디어에서 출발한 방법이다.

# 6.3.1 배치 정규화 알고리즘

배치 정규화는 2015년에 제안된 방법이다. 배치 정규화는 아직 세상에 나온 지 얼마 안된 기법임에도 많응ㄴ 연구자와 기술자가 즐겨 사용하고 있다. 실제로 기계학습 콘테스트의 결과를 보면 이 배치 정규화를 사용하여 뛰어난 결과를 달성한 예가 많다.

배치 정규화가 주목받는 이유는 다음과 같다.
- 학습을 빨리 진행할 수 있다(학습 속도 개선).
- 초깃값에 크게 의존하지 않는다.
- 오버피팅을 억제한다(드롭아웃 등의 필요성 감소).

딥러닝의 학습 시간이 길다는 걸 생각하면 첫 번째 이점은 아주 반가운 일이다. 초깃값에 크게 신경 쓸 필요가 없고, 오버피팅 억제 효과가 있다는 점도 딥러닝 학습의 두통거리를 덜어준다.

배치 정규화의 기본 아이디어는 앞에서 말했듯이 각 층에서의 활성화 값이 적당히 분포되도록 조정하는 것이다. 그래서 다음과 같이 데이터 분포를 정규화하는 "배치 정규화(Batch Norm) 계층"을 신경망에 삽입한다.

<img src=images/6_16.png height=200px width=600px>

배치 정규화는 그 이름과 같이 학습 시 미니배치를 단위로 정규화한다. 구체적으로는 데이터 분포가 평균이 0, 분산이 1이 되도록 정규화한다. 수식으로는 다음과 같다.

$
\mu_B \gets \frac{1}{m}\sum^{m}_{i=1} x_i \\
\sigma_{B}^{2} \gets \frac{1}{m}\sum^{m}_{i=1} (x_i - \mu_B)^2 \\
\hat{x_i} \gets \frac{x_i - \mu_B}{\sqrt{\sigma_{B}^{2}+\epsilon}}
\qquad$ [식 6.7]

여기에는 미니배치 $B = \left\{x_1, x_2, ..., x_m\right\}$이라는 m개의 입력 데이터의 집합에 대해 평균 $\mu_B$와 분산 $\sigma_{B}^{2}$을 구한다. 그리고 입력 데이터를 평균이 0, 분산이 1이 되도록 정규화한다. 여기서 $\epsilon$기호는 작은값 (예컨대 1e-7) 으로, 0으로 나누는 사태를 예방하는 역할을 한다.

[식 6.7]은 단순히 미니배치 입력 데이터 $\left\{x_1, x_2, ..., x_m\right\}$을 평균 0, 분산 1인 데이터 $\left\{\hat{x_1}, \hat{x_2}, ..., \hat{x_m}\right\}$으로 변환하는 일을 한다. 이 처리를 활성화 함수의 앞에 삽입함으로써 데이터 분포가 덜 치우치게 할 수 있다.

또, 배치 정규화 계층마다 이 정규화도니 데이터에 고유한 scale과 shift 변환을 수행한다. 수식으로는 다음과 같다.

$
y_i \gets \gamma\hat{x_i} + \beta
\qquad$ [식 6.8]

이 식에서 $\gamma$가 scale을, $\beta$가 shift를 담당한다. 두 값은 처음에는 $\gamma\.=\.1$, $\beta\.=\.0$부터 시작하고, 학습하면서 적합한 값으로 조정해간다.

이상이 배치 정규화의 알고리즘이다. 이 알고리즘이 신경망에서 순전파 때 적용된다. 이를 5장에서 설명한 계산 그래프로는 다음과 같이 그릴 수 있다.

<img src=images/6_17.png height=100px width=500px>

배치 정규화의 역전파 유도는 다소 복잡하므로 여기에서는 설명하지 않는다만, 계산 그래프를 그려보면 비교적 쉽게 도출할 수 있다. 자세한 설명은 프레드릭 크레저트(Frederik Kratzert)의 블로그에서 찾을 수 있다.

# 6.3.2 배치 정규화의 효과

배치 정규화 계층을 사용한 실험을 해보자. 우선은 MNIST 데이터셋을 사용하여 배치 정규화 계층을 사용할 때와 사용하지 않을 때의 학습 진도가 어떻게 달라지는지를 보자. 결과는 다음과 같다.

<img src=images/6_18.png height=100px width=500px>

다음과 같이 배치 정규화가 학습을 빨리 진전시키고 있다. 계속해서 초깃값 분포를 다양하게 줘가며 학습 진행이 어떻게 달라지는지 보자. 다음은 가중치 초깃값의 표준편차를 다양하게 바꿔가며 학습 경과를 관찰한 그래프이다.

<img src=images/6_19.png height=700px width=700px>

거의 모든 경우에서 배치 정규화를 사용할 때의 학습 진도가 빠른 것으로 나타난다. 실제로 배치 정규화를 이용하지 않는 경우엔 초깃값이 잘 분포되어 있지 않으면 학습이 전혀 진행되지 않는 모습도 보여준다.

지금까지 사렾본 것처럼 배치 정규화를 사용하면 학습이 빨라지며, 가중치 초깃값에 크게 의존하지 않아도 된다. 배치 정규화는 이처럼 장점이 많으니 앞으로 다양한 분야에서 활약할 것이다.